
# Hồi quy tuyến tính với SGD (Linear / Ridge / Lasso / ElasticNet)  
**Không dùng `country_name`, `country_code`** • **Ghi log RMSE/MAE/R² theo epoch** • **Early stopping**

---

## Bước 0 — Mục tiêu tổng quan
- Xây dựng pipeline hồi quy tuyến tính dựa trên **SGDRegressor** (4 biến thể: Linear, Ridge, Lasso, ElasticNet).
- **Loại bỏ** các cột định danh (`country_name`, `country_code`) vì **không mang tính dự báo** và dễ gây **overfitting**.
- Chỉ dùng **biến số (numeric)**; không chuẩn hoá target `life_expectancy`.
- Huấn luyện theo **epochs** bằng `partial_fit` để theo dõi **RMSE / MAE / R²** (Train/Val) qua từng epoch.
- Áp dụng **Early stopping** để dừng khi Val RMSE không cải thiện.
- Lưu **model `.pkl`**, **CSV** lịch sử metrics, **biểu đồ** RMSE/MAE/R² theo epoch.



## Bước 1 — Thiết lập & Đường dẫn
**Mục tiêu**
- Khai báo thư mục dữ liệu/đầu ra.
- Import các thư viện cần thiết.

**Hướng dẫn**
- Thay `DATA_DIR` nếu cấu trúc dự án khác.
- Thư mục `model/1_linear_regression` sẽ chứa `.pkl`, `.csv`, `.png`.


In [1]:

import os, joblib, numpy as np, pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import SGDRegressor
from sklearn.base import clone

DATA_DIR = "../data/processed"
X_TRAIN_PATH = f"{DATA_DIR}/X_train.csv"
Y_TRAIN_PATH = f"{DATA_DIR}/y_train.csv"
X_TEST_PATH  = f"{DATA_DIR}/X_test.csv"
Y_TEST_PATH  = f"{DATA_DIR}/y_test.csv"

MODEL_DIR = "../model/1_linear_regression"
os.makedirs(MODEL_DIR, exist_ok=True)
print("Paths ready.")
import numpy as np
import pickle  # use pickle.dumps/loads for snapshots


Paths ready.


In [2]:
def rmse_score(y_true, y_pred):
    """Return RMSE; no 'squared=False' to avoid version issues."""
    mse = mean_squared_error(y_true, y_pred)
    return float(np.sqrt(mse))

def compute_metrics(y_true, y_pred):
    return {
        'rmse': rmse_score(y_true, y_pred),
        'mae':  mean_absolute_error(y_true, y_pred),
        'r2':   r2_score(y_true, y_pred),
    }



## Bước 2 — Đọc dữ liệu & **Loại bỏ** cột định danh
**Mục tiêu**
- Loại `country_name`, `country_code` khỏi X vì đây là **ID** không mang tín hiệu dự báo.
- Chỉ giữ **numeric features**; không chuẩn hoá `life_expectancy` (target).

**Hướng dẫn**
- Dùng `.drop(..., errors='ignore')` để an toàn nếu cột không tồn tại.
- Nếu không có file Test, tách Validation từ Train theo tỉ lệ 80/20.


In [3]:

X_train = pd.read_csv(X_TRAIN_PATH)
y_train = pd.read_csv(Y_TRAIN_PATH).squeeze("columns")

HAS_TEST = os.path.exists(X_TEST_PATH) and os.path.exists(Y_TEST_PATH)
if HAS_TEST:
    X_test = pd.read_csv(X_TEST_PATH)
    y_test = pd.read_csv(Y_TEST_PATH).squeeze("columns")

# Bỏ cột định danh
id_cols = ['country_name', 'country_code']
X_train = X_train.drop(columns=id_cols, errors='ignore')
if HAS_TEST:
    X_test = X_test.drop(columns=id_cols, errors='ignore')

# Chỉ giữ numeric
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
X_train = X_train[num_cols].copy()
if HAS_TEST:
    X_test = X_test[num_cols].copy()

# Tạo tập validation nếu không có test
if not HAS_TEST:
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    print("Train:", X_train.shape, "| Val:", X_val.shape)
else:
    X_val, y_val = X_test, y_test
    print("Train:", X_train.shape, "| Test as Val:", X_val.shape)


Train: (4882, 12) | Test as Val: (543, 12)



## Bước 3 — Huấn luyện theo **Epochs** & Ghi log **RMSE/MAE/R²**
**Mục tiêu**
- Huấn luyện mô hình theo nhiều epoch để theo dõi hội tụ.
- Lưu model + lịch sử metrics + biểu đồ.

**Hướng dẫn**
- `partial_fit` trên toàn bộ batch mỗi epoch.
- Dừng sớm nếu Val RMSE không cải thiện sau `patience` epoch.
- Lưu kèm `num_cols` để đảm bảo thứ tự cột khi dự đoán sau này.


In [4]:

def epoch_metrics(y_true, y_pred):
    # Compatibility: some sklearn versions don't accept the 'squared' kwarg.
    try:
        rmse = rmse_score(y_true, y_pred)
    except TypeError:
        rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
    mae = mean_absolute_error(y_true, y_pred)
    r2  = r2_score(y_true, y_pred)
    return rmse, mae, r2

def train_with_epochs(model_name, sgd, Xtr, ytr, Xva, yva,
                      epochs=60, model_dir=MODEL_DIR,
                      early_stopping=True, patience=10):
    est = clone(sgd)
    best_val_rmse = np.inf
    best_state = None
    bad_epochs = 0

    hist = {"epoch": [], "train_rmse": [], "val_rmse": [],
            "train_mae": [], "val_mae": [], "train_r2": [], "val_r2": []}

    for ep in range(1, epochs+1):
        est.partial_fit(Xtr, ytr)
        ytr_pred = est.predict(Xtr)
        yva_pred = est.predict(Xva)

        # --- Metrics ---
        try:
            rmse_tr = rmse_score(ytr, ytr_pred)
            
        except TypeError:
            rmse_tr = float(np.sqrt(mean_squared_error(ytr, ytr_pred)))
        try:
            rmse_va = rmse_score(yva, yva_pred)
        except TypeError:
            rmse_va = float(np.sqrt(mean_squared_error(yva, yva_pred)))

        mae_tr  = mean_absolute_error(ytr, ytr_pred)
        mae_va  = mean_absolute_error(yva, yva_pred)
        r2_tr   = r2_score(ytr, ytr_pred)
        r2_va   = r2_score(yva, yva_pred)

        hist["epoch"].append(ep)
        hist["train_rmse"].append(rmse_tr); hist["val_rmse"].append(rmse_va)
        hist["train_mae"].append(mae_tr);   hist["val_mae"].append(mae_va)
        hist["train_r2"].append(r2_tr);     hist["val_r2"].append(r2_va)

        if early_stopping:
            if rmse_va + 1e-9 < best_val_rmse:
                best_val_rmse = rmse_va
                best_state = pickle.dumps(est)
                bad_epochs = 0
            else:
                bad_epochs += 1
                if bad_epochs >= patience:
                    if best_state is not None:
                        est = pickle.loads(best_state)
                    break

    # Save model bundle
    bundle = {"num_cols": num_cols, "estimator": est}
    pkl_path = os.path.join(model_dir, f"{model_name}.pkl")
    joblib.dump(bundle, pkl_path)

    # Save metrics CSV
    df = pd.DataFrame(hist)
    csv_path = os.path.join(model_dir, f"{model_name}_metrics.csv")
    df.to_csv(csv_path, index=False)

    # Plots
    x = df["epoch"].values
    plt.figure(); plt.plot(x, df["train_rmse"], label="Train RMSE"); plt.plot(x, df["val_rmse"], label="Val RMSE")
    plt.xlabel("Epoch"); plt.ylabel("RMSE"); plt.title(f"{model_name}: RMSE vs Epoch"); plt.legend()
    rmse_png = os.path.join(model_dir, f"{model_name}_rmse.png"); plt.savefig(rmse_png, bbox_inches="tight"); plt.close()

    plt.figure(); plt.plot(x, df["train_mae"], label="Train MAE"); plt.plot(x, df["val_mae"], label="Val MAE")
    plt.xlabel("Epoch"); plt.ylabel("MAE"); plt.title(f"{model_name}: MAE vs Epoch"); plt.legend()
    mae_png = os.path.join(model_dir, f"{model_name}_mae.png"); plt.savefig(mae_png, bbox_inches="tight"); plt.close()

    plt.figure(); plt.plot(x, df["train_r2"], label="Train R²"); plt.plot(x, df["val_r2"], label="Val R²")
    plt.xlabel("Epoch"); plt.ylabel("R²"); plt.title(f"{model_name}: R² vs Epoch"); plt.legend()
    r2_png = os.path.join(model_dir, f"{model_name}_r2.png"); plt.savefig(r2_png, bbox_inches="tight"); plt.close()

    return {"pkl": pkl_path, "csv": csv_path, "rmse_png": rmse_png, "mae_png": mae_png, "r2_png": r2_png,
            "last": {k: df[k].iloc[-1] for k in df.columns if k != "epoch"}}



## Bước 4 — Khai báo mô hình & Huấn luyện
**Mục tiêu**
- So sánh 4 biến thể: Linear / Ridge(L2) / Lasso(L1) / ElasticNet(L1+L2).
- Ghi log và lưu artifacts cho từng mô hình.

**Hướng dẫn**
- Với `sklearn>=1.0`, dùng `loss="squared_error"`; nếu bản rất cũ, chuyển thành `squared_loss`.


In [5]:

lin     = SGDRegressor(loss="squared_error", penalty=None,        learning_rate="constant", eta0=0.01, random_state=0)
ridge   = SGDRegressor(loss="squared_error", penalty="l2",        alpha=1e-4, learning_rate="constant", eta0=0.01, random_state=0)
lasso   = SGDRegressor(loss="squared_error", penalty="l1",        alpha=1e-5, learning_rate="constant", eta0=0.01, random_state=0)
elastic = SGDRegressor(loss="squared_error", penalty="elasticnet",alpha=1e-4, l1_ratio=0.15,
                       learning_rate="constant", eta0=0.01, random_state=0)

models = {
    "linear_sgd": lin,
    "ridge_sgd": ridge,
    "lasso_sgd": lasso,
    "elastic_sgd": elastic,
}

results = {}
for name, est in models.items():
    out = train_with_epochs(name, est, X_train.values, y_train.values, X_val.values, y_val.values,
                            epochs=10, early_stopping=True, patience=10)
    results[name] = out
    last = out["last"]
    print(f"{name}: RMSE tr/va={last['train_rmse']:.4f}/{last['val_rmse']:.4f} | "
          f"MAE tr/va={last['train_mae']:.4f}/{last['val_mae']:.4f} | "
          f"R² tr/va={last['train_r2']:.4f}/{last['val_r2']:.4f}")


linear_sgd: RMSE tr/va=3472797358335321.5000/3472921075720552.5000 | MAE tr/va=3472772375100840.0000/3472898787460626.0000 | R² tr/va=-160928700620656356082847842304.0000/-177032623699756806071012818944.0000
ridge_sgd: RMSE tr/va=3474329563172000.0000/3474773814040101.5000 | MAE tr/va=3474264706229455.5000/3474715463786253.5000 | R² tr/va=-161070736047919029939244892160.0000/-177221561258781300954163576832.0000
lasso_sgd: RMSE tr/va=20162351643359932.0000/20163302937068160.0000 | MAE tr/va=20162220194792032.0000/20163182735411364.0000 | R² tr/va=-5424465974595249087246753595392.0000/-5967423062453697765353002631168.0000
elastic_sgd: RMSE tr/va=3479361144414727.0000/3479419830150833.5000 | MAE tr/va=3479335559372553.0000/3479395957086889.0000 | R² tr/va=-161537604447208248261803507712.0000/-177695793382882339119482011648.0000


In [6]:

# Example to use later:
# bundle = joblib.load("model/linear_sgd.pkl")
# preprocess_loaded = bundle["preprocess"]
# estimator_loaded = bundle["estimator"]
# X_new should be a DataFrame with the same columns as X_train
# y_pred = estimator_loaded.predict(preprocess_loaded.transform(X_new))
# print("Sample prediction:", y_pred[:5])
print("To use later: joblib.load('../model/1_linear_regression/<name>.pkl') and call estimator.predict(preprocess.transform(X_new)).")


To use later: joblib.load('../model/1_linear_regression/<name>.pkl') and call estimator.predict(preprocess.transform(X_new)).
